In [44]:
import networkx as nx
import pandas as pd
import pickle
import numpy as np
import geopy.distance
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import osmnx as ox
import networkx as nx
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap as Basemap

Project_Path='/Users/nickkarras/PycharmProjects/ParkingViolationPredictionGraph_Git'

# Import Basic Infos

In [34]:
Sectors=pd.read_csv(Project_Path+ '/Data/Parking_Slot_Proccesed.csv',sep=',', index_col=0)
Nodelist=Sectors[['Slot_id','Latitude','Longitude']]
Legal_Ilegan=pd.read_csv(Project_Path+ '/Data/Scan_Data_Reg_2.3.csv',sep=',',index_col=0)
Unique_Sectors=Legal_Ilegan['Slot_id'].unique()
Unique_Sectors=pd.DataFrame(Unique_Sectors, columns=['Slot_id'])
Nodelist=pd.merge(Unique_Sectors,Nodelist, on='Slot_id')

with open(Project_Path+'Data/Dataset_Graph.pkl', 'rb') as inp:
    Train_Dataset = pickle.load(inp)
with open(Project_Path+'Data/Test_Dataset_Graph.pkl', 'rb') as inp:
    Test_Dataset = pickle.load(inp)

# Find Duration using API

In [35]:
import json
import openrouteservice

def get_Duration(long1,lat1,long2,lat2):
    coords = ((long1,lat1),(long2,lat2))
    client = openrouteservice.Client(key='test')
    res = client.directions(coords)
    #set location coordinates in longitude,latitude order

    #call API
    res = client.directions(coords)
    #test our response
    with(open(Project_Path+'Data/ETA.json','+w')) as f:
        f.write(json.dumps(res,indent=4, sort_keys=True))


    f = open('ETA.json')

    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    # Iterating through the json
    # list
    duration=data['routes'][0]['segments'][0]['duration']
    #Edge_Weights.append(Time_Distance)

    # Closing file
    f.close()
    return duration

In [36]:
def Normalize_Edgedata(Targets):
    Targets = (((0.9-0.1) * (Targets - np.min(Targets))) / (np.max(Targets) - np.min(Targets))) + 0.1
    return Targets

# Find All Combinations

In [37]:
def Get_Neighbors(Nodelist): 
    List=Nodelist.values.tolist()
    x=[]
    for i in range (0,len(List)):
        for k in range (0,len(List)):
             if List[i][0]!=List[k][0]:
                ed=[]
                ed.append(List[i])
                ed.append(List[k])
                x.append(ed)

    Edge_W=[]
    for i in tqdm(range (0,len(x))):
        Distance=[]
        b= geopy.distance.geodesic((x[i][1][1],x[i][1][2]), (x[i][0][1],x[i][0][2])).m
        if b<250:

            d=get_Duration(x[i][1][2],x[i][1][1], x[i][0][2],x[i][0][1])
            Distance.append(x[i][0][0])
            Distance.append(x[i][1][0])
            Distance.append(d)
            Edge_W.append(Distance)
    return Edge_W
    

In [38]:
def Preprocess_Weights(Edge_W):
    Weights=pd.DataFrame(Edge_W, columns=['Sector_1','Sector_2','Duration'])
    Weights['Sector_2']=Weights['Sector_2'].astype(int)
    Weights['Sector_1']=Weights['Sector_1'].astype(int)
    Weights['Duration']=Normalize_Edgedata(a['Duration'])
    Weights.Duration= 1-Weights.Duration
    return Weights

# Add self-edges

In [39]:
def Add_Self_Edges(Weights,Nodelist):
    Comb=[]
    N_List=Nodelist.values.tolist()
    for i in range (0,len(N_List)):
        Comb.append(N_List[i][0])
    Merged=pd.DataFrame(Comb,columns=['Sector_1'])
    Merged['Sector_2']=Comb
    Merged['Duration']=1
    
    Edgelist = pd.concat([Weights,Merged])
    
    Edgelist=Edgelist.reset_index()
    Edgelist=Edgelist.drop(['index'], axis=1)
    Edgelist['Sector_1']=Edgelist['Sector_1'].astype(int)
    Edgelist['Sector_2']=Edgelist['Sector_2'].astype(int)
    return Edgelist

# Prepare edges for model

In [40]:
def Get_Final_Edges(Edgelist,Train_Dataset,Test_Dataset):
    Slot_idx = {name: idx for idx, name in enumerate(sorted(Train_Dataset[6]["Slot_id"].unique()))}
    for i in tqdm (range(0,len(Train_Dataset))):
        Train_Dataset[i]["Slot_id"] = Train_Dataset[i]["Slot_id"].apply(lambda name: Slot_idx[name])

    for j in tqdm (range(0,len(Test_Dataset))):
        Test_Dataset[j]["Slot_id"] = Test_Dataset[j]["Slot_id"].apply(lambda name: Slot_idx[name])

    Edgelist["Sector_1"] = Edgelist["Sector_1"].apply(lambda name: Slot_idx[name])
    Edgelist["Sector_2"] = Edgelist["Sector_2"].apply(lambda name: Slot_idx[name])
    return Edgelist

In [41]:
def Graph_Edges(Nodelist,Train_Dataset,Test_Dataset):
    Edge_W=Get_Neighbors(Nodelist)
    Weights=Preprocess_Weights(Edge_W)
    Edgelist=Add_Self_Edges(Weights,Nodelist)
    Edgelist=Get_Final_Edges(Edgelist,Train_Dataset,Test_Dataset)
    return Edgelist

In [43]:
Edgelist=Graph_Edges(Nodelist,Train_Dataset,Test_Dataset)
Edgelist.to_csv(Project_Path+ '/Data/EdgesDuration250.csv')

100%|██████████| 49062/49062 [00:03<00:00, 12928.79it/s]


,Sector_1,Sector_2,Duration
0,10,11,0.759558
1,10,17963,0.330508
2,10,18143,0.586974
3,10,18144,0.521837
4,10,18224,0.542258
...,...,...,...
5833,93,93,1.000000
5834,94,94,1.000000
5835,95,95,1.000000
5836,96,96,1.000000
